In [2]:
#Slightly modified from ChatGPT

class Node:
    def __init__(self, key, weight=1, height=0, rightness = 0, left=None, right=None, parent=None, head=None, tail=None):
        self.key = key
        self.weight = weight
        self.height = height
        self.grosscost = key
        self.grossmin = key
        self.rightness = 0
        self.netcost = 0
        self.netmin = 0
        self.left = left
        self.right = right
        self.parent = parent
        self.head = head
        self.tail = tail
        self.reversed = False

class WeightBalancedTree:
    def __init__(self):
        self.root = None

    def weight(self, node):
        return 0 if node is None else node.weight

    def height(self, node):
        return -1 if node is None else node.height

    def update_weight(self, node):
        if node is not None:
            node.weight = 1 + self.weight(node.left) + self.weight(node.right)

    def update_height(self, node, current_height=0):
        if node is not None:
            node.height = current_height
            self.update_height(node.left, current_height + 1)
            self.update_height(node.right, current_height + 1)

    def update_head_tail(self, node):
        if node is not None:
            node.head = node if node.left is None else node.left.head
            node.tail = node if node.right is None else node.right.tail

    def update_parent(self, node, parent):
        if node is not None:
            node.parent = parent
            
    def update_rightness(self, node):
        if node is not None:
            node.rightness = node.parent.rightness + 1 if node.parent and node is node.parent.right else node.parent.rightness - 1 if node.parent and node is node.parent.left else 0
            self.update_rightness(node.left)
            self.update_rightness(node.right)

    def rotate_left(self, x):
        y = x.right
        x.right = y.left
        y.left = x
        self.update_weight(x)
        self.update_weight(y)
        self.update_height(x)
        self.update_height(y)
        self.update_head_tail(x)
        self.update_head_tail(y)

        # Update parent pointers
        y.parent = x.parent
        x.parent = y
        if x.right:
            x.right.parent = x
        self.update_rightness(x)
        self.update_rightness(y)
        # Update gross values
        self.update_gross(x)
        self.update_gross(y)

        return y

    def rotate_right(self, y):
        x = y.left
        y.left = x.right
        x.right = y
        self.update_weight(y)
        self.update_weight(x)
        self.update_height(y)
        self.update_height(x)
        self.update_head_tail(y)
        self.update_head_tail(x)

        # Update parent pointers
        x.parent = y.parent
        y.parent = x
        if y.left:
            y.left.parent = y
        self.update_rightness(x)
        self.update_rightness(y)
        # Update gross values
        self.update_gross(x)
        self.update_gross(y)

        return x


    def balance(self, root):
        if root is None:
            return root

        if self.weight(root.left) > 2 * self.weight(root.right):
            if self.weight(root.left.left) < self.weight(root.left.right):
                root.left = self.rotate_left(root.left)
            root = self.rotate_right(root)

        elif self.weight(root.right) > 2 * self.weight(root.left):
            if self.weight(root.right.right) < self.weight(root.right.left):
                root.right = self.rotate_right(root.right)
            root = self.rotate_left(root)

        # Update gross values after balancing
        self.update_rightness(root)
        self.update_gross(root)

        return root
    

    def update_gross(self, node):
        if node is not None and node.left is not None and node.right is not None:  # Ensure non-leaf node
            node.grosscost = min(node.key, node.left.grosscost, node.right.grosscost)
            node.grossmin = min(node.grosscost, node.left.grossmin, node.right.grossmin)
        else:  # Leaf node or null node
            node.grosscost = node.key #float('inf')
            node.grossmin = node.key #float('inf')

    def insert(self, root, key, parent=None, current_height=0):
        if root is None:
            new_node = Node(key, parent=parent, height=current_height)
            return new_node

        if key < root.key:
            root.left = self.insert(root.left, key, parent=root, current_height=current_height + 1)
        elif key > root.key:
            root.right = self.insert(root.right, key, parent=root, current_height=current_height + 1)

        self.update_weight(root)
        self.update_height(root, current_height)
        self.update_head_tail(root)
        root = self.balance(root)

        return root

    def insert_key(self, key):
        self.root = self.insert(self.root, key)

    def inorder_traversal(self, node, result):
        if node is not None:
            self.inorder_traversal(node.left, result)
            result.append({'key' : node.key, 'rightness': node.rightness, 'grosscost': node.grosscost, 'grossmin': node.grossmin, 'left': node.left.key if node.left else None, 'right': node.right.key if node.right else None, 'weight': node.weight, 'height': node.height, 'head': node.head.key if node.head else None, 'tail': node.tail.key if node.tail else None, 'parent': node.parent.key if node.parent else None})
            self.inorder_traversal(node.right, result)

    def display(self):
        result = []
        self.inorder_traversal(self.root, result)
        return result

# Example usage:
if __name__ == "__main__":
    weight_balanced_tree = WeightBalancedTree()
    keys = [9, 5, 10, 0, 6, 11, -1, 1, 2, 5.5]

    for key in keys:
        weight_balanced_tree.insert_key(key)

    print("Inorder Traversal:", weight_balanced_tree.display())


Inorder Traversal: [{'key': -1, 'rightness': -2, 'grosscost': -1, 'grossmin': -1, 'left': None, 'right': None, 'weight': 1, 'height': 2, 'head': -1, 'tail': -1, 'parent': 0}, {'key': 0, 'rightness': -1, 'grosscost': -1, 'grossmin': -1, 'left': -1, 'right': 2, 'weight': 4, 'height': 1, 'head': -1, 'tail': 2, 'parent': 5}, {'key': 1, 'rightness': -1, 'grosscost': 1, 'grossmin': 1, 'left': None, 'right': None, 'weight': 1, 'height': 3, 'head': 1, 'tail': 1, 'parent': 2}, {'key': 2, 'rightness': 0, 'grosscost': 2, 'grossmin': 2, 'left': 1, 'right': None, 'weight': 2, 'height': 2, 'head': 1, 'tail': 2, 'parent': 0}, {'key': 5, 'rightness': 0, 'grosscost': -1, 'grossmin': -1, 'left': 0, 'right': 9, 'weight': 10, 'height': 0, 'head': -1, 'tail': 11, 'parent': None}, {'key': 5.5, 'rightness': 0, 'grosscost': 5.5, 'grossmin': 5.5, 'left': None, 'right': 6, 'weight': 2, 'height': 2, 'head': 5.5, 'tail': 6, 'parent': 9}, {'key': 6, 'rightness': 1, 'grosscost': 6, 'grossmin': 6, 'left': None, 'rig

In [4]:
class DynamicTree:
    
    def path(self, v):
        w = v
        while w.parent != None:
            w = w.parent
        return w

    def head(self, p):
        if p.reversed:
            return p.tail
        return p.head

    def before(self, v):
        root = self.path(v)
        head_v = self.head(root)
        if v == head_v:
            return None
        u = head_v
        while u.parent != v:
            if u == root:
                break  
            else:
                u = u.parent
        if u.parent == v:
            return u
        else:
            return v.parent

    def after(self, v):
        root = self.path(v)
        tail_v = self.tail(root)
        if v == tail_v:
            return None
        u = tail_v
        while u.parent != v:
            if u == root:
                break
            else:
                u = u.parent

        if u.parent == v:
            return u
        else:
            return v.parent


    def pcost(self, v):
        nodes = []
        w = v
        deepest_left_child = v
        while w.parent != None:
            nodes.append(w.parent)
            w = w.parent
        for i in range(len(nodes) - 1, 1, -1):
            if nodes[i].left == nodes[i-1]:
                deepest_left_child = nodes[i-1]
        return deepest_left_child.parent.grosscost

    def tail(self, p):
        if p.reversed:
            return p.head
        return p.tail

    def reverse(self, p):
        p.reversed = not p.reversed
        #temp_head = p.head
        #p.head = p.tail
        #p.tail = p.temp_head
        temp_left = p.left
        p.left = p.right
        p.right = p.temp_left
        return p

    def pupdate(self, p, x):
        p.netmin += x
        return p

    def pmincost(self, p):
        u = p
        while True:
            if (u.netcost == 0 and (u.right.netmin > 0 or (u.right.right == None and u.right.left == None))):
                break
            elif u.right.netcost == 0 and (u.right.right != None or u.right.left != None):
                u = u.right
            elif u.netcost > 0:
                u = u.left
        if u.left == None and u.right == None:
            return u
        if u.reversed:
            return u.head
        return u.tail

    def construct(self, v, w, x):
        root = Node()
        root.grosscost = x
        root.left = v
        root.right = w
        return root

    def destroy(self, u):
        v = u.left
        w = u.right
        x = u.grosscost
        del u
        return [v, w, x]
    
    def split(self, v):
        w = self.path(v)
        head_v = self.head(w)
        tail_v = self.tail(w)
        before_v = self.before(v)
        after_v = self.after(v)
        x = before_v.key
        y = v.key
        p = before_v
        q = after_v

        return [p, q, x, y]

    def concatenate(self, p, q, x):
        v = self.tail(p)
        w = self.head(q)
        return self.construct(v, w, x)

    def dparent(self, v):
        return v.parent

    def splice(self, p):
        v = self.dparent(self.tail(p))
        q, r, x, y = self.split(v)
        if q != None:
            self.tail(q).parent, self.dcost(tail(q)) = v, x
        p = self.concatenate(p, self.path(v), self.dcost(self.tail(p)))
        if r == None:
            return p
        return self.concatenate(p, r, y)

    def expose(self, v):
        q, r, x, y = self.split(v)
        if q != None:
            self.dparent(self.tail(q)), dcost(self.tail(q)) = v, x
        if r == None:
            p = self.path(v)
        else:
            p = self.concatenate(self.path(v), r, y)
        while self.dparent(self.tail(p)) != None:
            p = self.splice(p)
        return p

    def parent(self, v):
        if v == self.tail(path(v)):
            return self.dparent(v)
        return self.after(v)

    def root(self, v):
        return self.tail(self.expose(v))

    def cost(self, v):
        if v == self.tail(self.path(v)):
            return self.dcost(v)
        else:
            return self.pcost(v)

    def vertex_mincost(self, v):
        return self.mincost(self.expose(v))

    def vertex_update(self, v, x):
        self.update(self.expose(v), x)

    def link(self, v, w, x):
        self.concatenate(self.path(v), self.expose(w), x)

    def cut(self, v):
        self.expose(v)
        p, q, x, y = self.split(v)
        v.parent = None
        return y

    def evert(self, v):
        self.reverse(self.expose(v))
        v.parent = None

SyntaxError: cannot assign to function call (1746644175.py, line 133)